Imports

In [2]:
from ga_polynomial import createPopulation, grade, createIndividual, evolve, fitness
import plotly.express as px
from math import ceil
import numpy as np
import json
import plotly.graph_objects as go
import logging
from tqdm import tqdm
import matplotlib as plt

In [20]:
def getMSE(individual):
    """
    Take in individual, which are effectively coefficient predictions, then use a range of x values to generate a corresponding y value
    with the same equation pattern as the polynomial we are trying to fit. Return the mean square error by comparing with y values
    generated from target polynomial.
    :param individual:
    :return:
    """

    # Extract polynomial coefficients from individual
    a1 = individual[0]
    a2 = individual[1]
    a3 = individual[2]
    a4 = individual[3]
    a5 = individual[4]
    c = individual[5]

    yTarget = np.array([])
    yPredict = np.array([])

    # For a range of values of x, calculate the target and predicted value of y
    for x in np.linspace(-1,1, 20):
        np.append(yTarget, 25*x**5 + 18*x**4 + 0 - 14*x**2 + 7*x -19)
        np.append(yPredict, a1*x**5 + a2*x**4 + a3*x**3 + a4*x**2 + a5*x + c)

    # Calculate error as difference between each y value prediction and corresponding target
    errors = np.subtract(yTarget, yPredict)

    # Square each error term
    squaredErrors = np.power(errors, 2)

    # Calculate weighted average of the squared errors array
    meanSquaredErrors = np.average(squaredErrors)

    assert isinstance(meanSquaredErrors, float)

    return meanSquaredErrors

In [35]:
import pandas as pd

In [63]:
population = createPopulation(100, -50, 50, 6)
population[:5]

[array([ 13,  39, -24,  -8,  29,  33]),
 array([ 12,  43,  40, -31,  12, -33]),
 array([-41,  -9, -27,   7, -26, -34]),
 array([-37, -22,  27,  29,  46,  18]),
 array([  8, -26, -41,  10,   2,  28])]

In [64]:
fitnesses = np.random.randint(0, 10, (100,1))
fitnesses[:5]

array([[2],
       [0],
       [8],
       [3],
       [7]])

In [65]:
for individual, fitness in zip(population, fitnesses):
    print(individual, fitness)
    break

[ 13  39 -24  -8  29  33] [2]


In [ ]:
population['mse'] = getMSE(np.array())

In [19]:
a = np.array([population, population**2]).T
#a = a.reshape(len(a[0]), 2)
a

array([[ 0,  0],
       [ 1,  1],
       [ 2,  4],
       [ 3,  9],
       [ 4, 16],
       [ 5, 25],
       [ 6, 36],
       [ 7, 49],
       [ 8, 64],
       [ 9, 81]])

In [24]:
np.array([population, lambda x: getMSE(x)])

array([array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
       <map object at 0x000001F6FF3FBF10>], dtype=object)

---

Setup

In [ ]:
count = 500
individualLength = 6
individualMin = -50
individualMax = 50

retain = 0.1
mutate = 0.01
random_select = 0.1

In [ ]:
maxGen = 1000
polynomials = {}
checkpoints = np.linspace(100, maxGen, int(maxGen/100))

p = population(count, individualMin, individualMax, individualLength)
fitness_history = [grade(p)]
lastGrade = 1
it = 0
x = np.linspace(-1,1, 20)

while lastGrade != 0 and len(fitness_history) < maxGen:

    p, individualGrades = evolve(p, 
               retain=retain, 
               random_select=random_select, 
               mutate=mutate)

    lastGrade = grade(p)
    fitness_history.append(lastGrade)
    
    if it in checkpoints:

        # Rank population based on individual grades
        bestIndividual = sorted(individualGrades)[0][1]
        
        # Extract polynomial coefficients from individual
        a1 = bestIndividual[0]
        a2 = bestIndividual[1]
        a3 = bestIndividual[2]
        a4 = bestIndividual[3]
        a5 = bestIndividual[4]
        c = bestIndividual[5]
        
        y = []
        
        # For a range of values of x, calculate the target and predicted value of y
        for element in x:
            yPredict = a1*element**5 + a2*element**4 + a3*element**3 + a4*element**2 + a5*element + c
            
            # Store target and prediction as pair in list of tuples
            y.append(yPredict)
        
        polynomials[it] = (x, np.array(y))
        
        figPolynomial = px.line(polynomials)
        figPolynomial.show()
        
    it += 1

#results[', '.join([str(retain), str(mutate), str(random_select)])] = filledArray

In [ ]:
np.array(y)

---

In [ ]:
fig = px.line(fitness_history)
fig.show()

In [ ]:
data = []

for key in results.keys():
    droppedNones = list(filter(None, results[key]))
    lowestVal = min(droppedNones)
    data.append((lowestVal, key))
    
bestPerformers = sorted(data)[:10]
bestPerformers

In [ ]:
maxIterations = 1000
resultsBest = {}

for _, params in bestPerformers:
    
    # Get params from list of best performers
    paramList = params.split(', ')
    retain = float(paramList[0])
    mutate = float(paramList[1])
    random_select = float(paramList[2])
    
    logging.warning("retain: {}; mutate: {}; random_select: {}\n".format(retain, mutate, random_select))
    
    # Create and grade first population
    p = population(count, individualMin, individualMax, individualLength)
    fitness_history = [grade(p)]
    lastGrade = 1

    # Until you do max iterations or reach error = 0
    while lastGrade != 0 and len(fitness_history) < maxIterations:

        # Create population
        p = evolve(p, 
                   retain=retain, 
                   random_select=random_select, 
                   mutate=mutate)

        # Grade population
        lastGrade = grade(p)
        fitness_history.append(lastGrade)

        # Fill with Nones to make all same length
        filledArray = [None]*maxIterations
        lastIndex = len(fitness_history)
        filledArray[:lastIndex-1] = fitness_history

    # Store population results in new dictionary for best performers (length 10)
    resultsBest[', '.join([str(retain), str(mutate), str(random_select)])] = filledArray

In [ ]:
with open("results_polynomial_broadSweep.json", "w") as outfile:  
    json.dump(results, outfile) 
with open("results_polynomial_bestPerformers.json", "w") as outfile:  
    json.dump(resultsBest, outfile) 

---

In [ ]:
import json
with open('results_polynomial_bestPerformers.json') as json_file: 
    resultsBest = json.load(json_file)

In [ ]:
import plotly.express as px
fig = px.line(resultsBest)
fig.show()